## <center>Analiza poziomu PM2.5 w afrykańskich miastach</center>
### by
<ol>
    <li style='font-size: 20px'>Hubert Kłosowski 242424</li>
    <li style='font-size: 20px'>Krzysztof Kolanek 242425</li>
    <li style='font-size: 20px'>Kamil Małecki 242464</li>
</ol>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler

### Wczytanie danych

In [ ]:
data = pd.read_csv('data\\Train.csv')
final_test = pd.read_csv('data\\Test.csv')

data.info()

In [ ]:
data.head()

### Rozbicie daty na składowe

In [ ]:
def change_date(df):
    data['date'] = pd.to_datetime(data['date'])
    df['day'] = df['date'].dt.dayofweek
    df['month'] = df['month'].astype(np.int64)
    df['year'] = df['date'].dt.year
    # df.drop(columns=['id', 'city', 'country', 'site_id', 'date'], inplace=True)
    return df

data = change_date(data)

### Wykres przedstawiający jakość powietrza z krajach afrykańskich

In [0]:
sns.lineplot(data=data, x='date', y='pm2_5', hue='country')
plt.title('Jakość powietrza z podziałem na kraje')

### Wykres przedstawiający wartość pm2_5 w zarejestrowanych godzinach

In [0]:
sns.barplot(data=data, x='hour', y='pm2_5', hue='country')
plt.title('Jakość powietrza w poszczególnych godzinach z podziałem na kraje')

### Wykres przedstawiający wartość pm2_5 z zależności od miesięcy i krajów

In [0]:
sns.barplot(data=data, x='month', y='pm2_5', hue='country')
plt.title('Jakość powietrza w poszczególnych miesiącach z podziałem na kraje')

### Korelacje poszczególnych grup kolumn

In [ ]:
def correlation():
    columns = data.columns.str.split('_', expand=True).levels[0].to_frame()
    columns.drop(['year', 'month', 'day', 'hour', 'pm2'], inplace=True)
    columns = columns[0].tolist()
    for i, el in enumerate(columns):
        fig, ax = plt.subplots(figsize=(10, 10))
        selected_columns = [col for col in data.columns if col.startswith(el) or col == 'pm2_5']
        sns.heatmap(data[selected_columns].corr(), annot=True, fmt='.2f', cmap='viridis', ax=ax)
        plt.tight_layout()
        plt.show()
    
correlation()

In [ ]:
data

In [ ]:
data.columns

In [ ]:
data['sulphurdioxide_so2_column_number_density'].describe()